# Import 

In [7]:
# !pip install --upgrade torch torchvision
!pip install torch==2.1.2 torchvision==0.16.2

import torch
import torchvision

print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)

PyTorch version: 2.1.2+cu121
Torchvision version: 0.16.2+cu121


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torchvision import transforms, datasets, models
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from torch.cuda import amp
from torch.nn.parameter import Parameter
from PIL import Image
import matplotlib.pyplot as plt
import os
import random
import torchvision
import numpy as np
import gc  
from albumentations import (
    Compose, Normalize, Resize,
    RandomResizedCrop, HorizontalFlip,
    RandomBrightnessContrast, ShiftScaleRotate
)
from albumentations.pytorch import ToTensorV2
import cv2
from datetime import datetime
import pytz
import cv2
import timm
import math
from tqdm import tqdm
import sys
from sklearn.metrics import roc_auc_score
from scipy.spatial.distance import cdist

sys.path.append('../tools')
from utils import *

In [9]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

# Use a chosen seed
set_seed(42)

In [10]:
# Check if CUDA is available and set PyTorch to use GPU or CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# 実験1.マルチクラス分類

## データの準備

In [11]:
# 必要なディレクトリ構成に変更

copy_images('../data/input/Hazelnut/train/good/','../data/input/01_multiclass/train/Hazelnut/')
copy_images('../data/input/Rotary_beacon_light/train/good/','../data/input/01_multiclass/train/Rotary_beacon_light/')
copy_images('../data/input/Coffee_beans/train/good/','../data/input/01_multiclass/train/Coffee_beans/')

copy_images('../data/input/Coffee_beans/test/candy/','../data/input/01_multiclass/test/Coffee_beans_abn/')
copy_images('../data/input/Coffee_beans/test/chocolate/','../data/input/01_multiclass/test/Coffee_beans_abn/')
copy_images('../data/input/Coffee_beans/test/good/','../data/input/01_multiclass/test/Coffee_beans_good/')

copy_images('../data/input/Hazelnut/test/crack/','../data/input/01_multiclass/test/Hazelnut_abn/')
copy_images('../data/input/Hazelnut/test/good/','../data/input/01_multiclass/test/Hazelnut_good/')

copy_images('../data/input/Rotary_beacon_light/test/good/','../data/input/01_multiclass/test/Rotary_beacon_light_good/')
copy_images('../data/input/Rotary_beacon_light/test/nocolor/','../data/input/01_multiclass/test/Rotary_beacon_light_abn/',prefix='nocolor')
copy_images('../data/input/Rotary_beacon_light/test/red/','../data/input/01_multiclass/test/Rotary_beacon_light_abn/',prefix='red')
copy_images('../data/input/Rotary_beacon_light/test/yellow/','../data/input/01_multiclass/test/Rotary_beacon_light_abn/',prefix='yellow')

Copied: ../data/input/Hazelnut/train/good/o039.jpg to ../data/input/01_multiclass/train/Hazelnut/o039.jpg
Copied: ../data/input/Hazelnut/train/good/o043.jpg to ../data/input/01_multiclass/train/Hazelnut/o043.jpg
Copied: ../data/input/Hazelnut/train/good/o019.jpg to ../data/input/01_multiclass/train/Hazelnut/o019.jpg
Copied: ../data/input/Hazelnut/train/good/o010.jpg to ../data/input/01_multiclass/train/Hazelnut/o010.jpg
Copied: ../data/input/Hazelnut/train/good/o004.jpg to ../data/input/01_multiclass/train/Hazelnut/o004.jpg
Copied: ../data/input/Hazelnut/train/good/o038.jpg to ../data/input/01_multiclass/train/Hazelnut/o038.jpg
Copied: ../data/input/Hazelnut/train/good/o029.jpg to ../data/input/01_multiclass/train/Hazelnut/o029.jpg
Copied: ../data/input/Hazelnut/train/good/o009.jpg to ../data/input/01_multiclass/train/Hazelnut/o009.jpg
Copied: ../data/input/Hazelnut/train/good/o011.jpg to ../data/input/01_multiclass/train/Hazelnut/o011.jpg
Copied: ../data/input/Hazelnut/train/good/o026

## 学習フェーズ

### 関数

In [13]:
def gem(x, p=3, eps=1e-6):
    return F.avg_pool2d(
        x.clamp(min=eps).pow(p),
        (x.size(-2), x.size(-1))
    ).pow(1./p)


class GeM(nn.Module):

    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return gem(x, p=self.p, eps=self.eps)

    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(
            self.p.data.tolist()[0]
        ) + ', ' + 'eps=' + str(self.eps) + ')'

    
class ArcMarginProduct(nn.Module):
    """Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=64.0, m=1.0, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.ls_eps = ls_eps  # label smoothing
        self.weight = Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        input = input.float()

        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        one_hot = torch.zeros(cosine.size(), device=device)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s

        return output


class AngularModel(nn.Module):

    def __init__(self, n_classes=2515, model_name="resnet34", pooling="GeM",
                 margin=0.3, scale=30, fc_dim=512,
                 pretrained=True, loss_kwargs=None):
        super(AngularModel, self).__init__()

        self.backbone = timm.create_model(
            model_name,
            pretrained=pretrained,
        )
        final_in_features = self.backbone.fc.in_features

        self.backbone = nn.Sequential(*list(self.backbone.children())[:-2])

        loss_kwargs = loss_kwargs or {
            "s": scale,
            "m": margin,
            "easy_margin": False,
            "ls_eps": 0.0,
        }

        self.pooling = GeM()
        self.dropout = nn.Dropout(p=0.0)
        self.fc = nn.Linear(final_in_features, fc_dim)
        self.bn = nn.BatchNorm1d(fc_dim)
        self._init_params()

        self.final = ArcMarginProduct(fc_dim,
                                      n_classes,
                                      **loss_kwargs)

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, x, label):
        feature = self.extract_features(x)
        logits = self.final(feature, label)
        return logits

    def extract_features(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = self.pooling(x).view(batch_size, -1)

        # fc
        x = self.dropout(x)
        x = self.fc(x)
        x = self.bn(x)

        return x

class CNNModel(nn.Module):
    def __init__(self, n_classes=2515, model_name="resnet34", fc_dim=512, pretrained=True):
        super(CNNModel, self).__init__()

        # バックボーンの初期化
        self.backbone = timm.create_model(model_name, pretrained=pretrained)
        final_in_features = self.backbone.fc.in_features

        # バックボーンの最終層を除去
        self.backbone = nn.Sequential(*list(self.backbone.children())[:-2])

        # グローバル平均プーリングと全結合層の追加
        self.pooling = GeM()
        self.dropout = nn.Dropout(p=0.0)
        self.fc = nn.Linear(final_in_features, fc_dim)
        self.bn = nn.BatchNorm1d(fc_dim)
        self.relu = nn.ReLU(inplace=True)
        self.classifier = nn.Linear(fc_dim, n_classes)

        self._init_params()

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)
        nn.init.xavier_normal_(self.classifier.weight)
        nn.init.constant_(self.classifier.bias, 0)

    def forward(self, x, label):
        feature = self.extract_features(x)
        x = self.relu(feature)

        # 分類器
        logits = self.classifier(x)
        return logits

    def extract_features(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = self.pooling(x).view(batch_size, -1)

        # fc
        x = self.dropout(x)
        x = self.fc(x)
        x = self.bn(x)

        return x

def get_transforms():
    transforms_dict = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(512),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]),
        'valid': transforms.Compose([
            transforms.Resize((512, 512)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]),
    }
    return transforms_dict

In [14]:
def train_model(dataloaders, model, criterion, optimizer, num_epochs, file_name):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            correct_predictions = 0
            
            # tqdmを追加して、ループの進捗を表示
            for inputs, labels in tqdm(dataloaders[phase], desc=f"{phase} epoch {epoch+1}"):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs, labels)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                correct_predictions += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = correct_predictions.double() / len(dataloaders[phase].dataset)
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    torch.save(model.state_dict(), f'../models/{file_name}.pth')
    print('Finished Training')

## 実行

In [15]:
def load_datasets(root_dir, transforms_dict):
    datasets = {
        'train': torchvision.datasets.ImageFolder(root=root_dir, transform=transforms_dict['train']),
        # 'test': torchvision.datasets.ImageFolder(root=root_dir, transform=transforms_dict['valid']),
    }
    return datasets

def get_dataloaders(datasets, batch_size):
    dataloaders = {
        'train': DataLoader(datasets['train'], batch_size=batch_size, shuffle=True),
        # 'test': DataLoader(datasets['test'], batch_size=batch_size, shuffle=False),
    }
    return dataloaders

In [16]:
# Get transforms
transforms_dict = get_transforms()

# Load datasets
datasets = load_datasets(root_dir='../data/input/01_multiclass/train/', transforms_dict=transforms_dict)

# Create dataloaders
dataloaders = get_dataloaders(datasets, batch_size=32)

# Define model
n_classes = 3
arcmodel = AngularModel(n_classes=n_classes)
arcmodel = arcmodel.to(device)

cnnmodel = CNNModel(n_classes=n_classes)
cnnmodel = cnnmodel.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
arc_optimizer = optim.SGD(arcmodel.parameters(), lr=0.001, momentum=0.9)
cnn_optimizer = optim.SGD(cnnmodel.parameters(), lr=0.001, momentum=0.9)

# Train the model
train_model(dataloaders, arcmodel, criterion, arc_optimizer, num_epochs=10, file_name='angularmodel')
train_model(dataloaders, cnnmodel, criterion, cnn_optimizer, num_epochs=10, file_name='cnnmodel')

Epoch 1/10
----------


train epoch 1: 100%|██████████| 5/5 [00:07<00:00,  1.58s/it]


train Loss: 9.1740 Acc: 0.0000
Epoch 2/10
----------


train epoch 2: 100%|██████████| 5/5 [00:06<00:00,  1.27s/it]


train Loss: 5.7524 Acc: 0.0200
Epoch 3/10
----------


...

train Loss: 0.0547 Acc: 0.9933
Epoch 9/10
----------


train epoch 9: 100%|██████████| 5/5 [00:06<00:00,  1.25s/it]


train Loss: 0.0599 Acc: 0.9933
Epoch 10/10
----------


train epoch 10: 100%|██████████| 5/5 [00:06<00:00,  1.25s/it]


train Loss: 0.0479 Acc: 1.0000
Finished Training


## 予測フェーズ

### 関数

In [17]:
def extract_vectors(model, dataloader, out_dim, device):
    with torch.no_grad():
        vecs = torch.zeros(out_dim, len(dataloader.dataset)).to(device)
        labels = []
        for i, (images, lbls) in enumerate(dataloader):
            images = images.to(device)
            vecs[:, i] =  model.extract_features(images).squeeze()
            labels.extend(lbls.numpy())
        return vecs, labels

# 新しいラベルの割り当て
def reassign_labels(class_to_idx):
    new_labels = {}
    for class_name, label in class_to_idx.items():
        if '_good' in class_name:
            new_labels[label] = 0
        elif '_abn' in class_name:
            new_labels[label] = 1
    return new_labels

def main(path, model,input_size=512, out_dim=512, ):
    
    # model = AngularModel(n_classes=n_classes)
    model.load_state_dict(torch.load(path))
    model.to(device)
    model.eval() 
    
    transforms_dict = get_transforms()

    # Define the ImageFolder datasets for index and query images
    train_dataset = torchvision.datasets.ImageFolder(root='../data/input/01_multiclass/train/', transform=transforms_dict['valid'])
    test_dataset = torchvision.datasets.ImageFolder(root='../data/input/01_multiclass/test/', transform=transforms_dict['valid'])
    
    # Define the DataLoaders
    train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=False, num_workers=2, pin_memory=True)
    test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=2, pin_memory=True)

    # Extract vectors
    train_vectors, train_labels = extract_vectors(model, train_dataloader, out_dim, device=device)
    test_vectors, test_labels = extract_vectors(model, test_dataloader, out_dim, device=device)

    # Convert vectors to numpy
    train_vectors = train_vectors.cpu().numpy()
    test_vectors = test_vectors.cpu().numpy()

    # 距離計算
    distances = cdist(test_vectors.T, train_vectors.T, 'cosine')
    distance_j = np.min(distances, axis=1)

    # ラベルを新しい値に変更
    new_class_to_idx = reassign_labels(class_to_idx) #'*_good'のlabel値(eg.0)に対して新しいlabel値 0を割り当て
    new_test_labels = [new_class_to_idx[label] for label in test_labels]

    auc_score = roc_auc_score(new_test_labels, distance_j)
    print('auc score: ', auc_score)
    # return train_vectors, train_labels, test_vectors, test_labels

### 実行

In [18]:
# !find ../data/input/ -name .ipynb_checkpoints -type d -exec rm -rf {} +

In [19]:
class_to_idx = {
    'Coffee_beans_abn': 0, 'Coffee_beans_good': 1, 'Hazelnut_abn': 2,
    'Hazelnut_good': 3, 'Rotary_beacon_light_abn': 4, 'Rotary_beacon_light_good': 5
}

if __name__ == "__main__":
    n_classes=3
    arcmodel = AngularModel(n_classes=n_classes)
    cnnmodel = CNNModel(n_classes=n_classes)

    print('■arcfaceによる推論結果')
    main(path='../models/angularmodel.pth', model=arcmodel)
    print('■CNNによる推論結果')
    main(path='../models/cnnmodel.pth', model=cnnmodel)

# print('train ',arc_train_vectors.shape, len(arc_train_labels))
# print('test: ',arc_test_vectors.shape, len(arc_test_labels))

■arcfaceによる推論結果
auc score:  0.9331174089068827
■CNNによる推論結果
auc score:  0.9072064777327935


## grad-camによるモデル判断根拠を可視化

In [20]:
!pip install grad-cam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.4 MB/s eta 0:00:00:00:010:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 29.3 MB/s eta 0:00:00:00:0100:01
  Created wheel for grad-cam: filename=grad_cam-1.5.0-py3-none-any.whl size=38070 sha256=7e75b376ee7be3c752728813b265d813664dfe97d1a15d797a75b3f4c538cd22
  Stored in directory: /home/jovyan/.cache/pip/wheels/fa/42/c0/201f854a89e14b4e7235cf0e5e0809dbdf3ae4f348e882405d
Successfully built grad-cam


In [21]:
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam import GradCAM

In [25]:
train_dataset = torchvision.datasets.ImageFolder(root='../data/input/01_multiclass/train/', transform=transforms_dict['valid'])
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=False, num_workers=2, pin_memory=True)

img, label = train_dataset[60]
# input_img = input_transform(img)
# img = img_transform(img)



NameError: name 'img_transform' is not defined

In [26]:
img

tensor([[[-1.3473, -1.3473, -1.3473,  ..., -1.3473, -1.3473, -1.3473],
         [-1.3473, -1.3473, -1.3473,  ..., -1.3473, -1.3473, -1.3473],
         [-1.3473, -1.3473, -1.3473,  ..., -1.3473, -1.3473, -1.3473],
         ...,
         [-1.3473, -1.3473, -1.3473,  ..., -1.3473, -1.3473, -1.3473],
         [-1.3473, -1.3473, -1.3473,  ..., -1.3473, -1.3473, -1.3473],
         [-1.3473, -1.3473, -1.3473,  ..., -1.3473, -1.3473, -1.3473]],

        [[-1.2479, -1.2479, -1.2479,  ..., -1.2479, -1.2479, -1.2479],
         [-1.2479, -1.2479, -1.2479,  ..., -1.2479, -1.2479, -1.2479],
         [-1.2479, -1.2479, -1.2479,  ..., -1.2479, -1.2479, -1.2479],
         ...,
         [-1.2479, -1.2479, -1.2479,  ..., -1.2479, -1.2479, -1.2479],
         [-1.2479, -1.2479, -1.2479,  ..., -1.2479, -1.2479, -1.2479],
         [-1.2479, -1.2479, -1.2479,  ..., -1.2479, -1.2479, -1.2479]],

        [[-1.0201, -1.0201, -1.0201,  ..., -1.0201, -1.0201, -1.0201],
         [-1.0201, -1.0201, -1.0201,  ..., -1

In [27]:
n_classes=3
model = AngularModel(n_classes=n_classes)
# model = AngularModel(n_classes=n_classes)
model.load_state_dict(torch.load('../models/angularmodel.pth'))
# model.to(device)
model.eval() 

AngularModel(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act1): ReLU(inplace=True)
        (aa): Identity()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=Fa

In [ ]:
'''
    実装途中
'''

# target_layers = [model.backbone[-3]]
# cam = GradCAM(
#     model=model, target_layers=target_layers, use_cuda=torch.cuda.is_available()
# )